In [1]:
from selenium import webdriver
google_web_driver_path = "/Users/ilkersenturk/Desktop/nlp/chromedriver"
    
driver = webdriver.Chrome(google_web_driver_path)
driver.get("http://www.lightnovelpub.com/novel/martial-peak-19072354/reviews")


book_reviews = []

reviews = driver.find_element_by_id("review_load")
for review in reviews.find_elements_by_class_name("review-text"):
    
    book_reviews.append(str(review.text))

driver.close()

/var/folders/ly/yk3m2zx54dn3f8s2ydnw765r0000gn/T/ipykernel_68879/434258147.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(google_web_driver_path)
/var/folders/ly/yk3m2zx54dn3f8s2ydnw765r0000gn/T/ipykernel_68879/434258147.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  reviews = driver.find_element_by_id("review_load")
/Users/ilkersenturk/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webelement.py:464: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [2]:
driver = webdriver.Chrome(google_web_driver_path)

driver.get("https://www.webnovel.com/book/system-in-martial-peak_19139372105400305/reviews")

reviews = driver.find_element_by_id("review")
for review in reviews.find_elements_by_class_name("j_book_review"):
    
    book_reviews.append(str(review.text))

driver.close()

/var/folders/ly/yk3m2zx54dn3f8s2ydnw765r0000gn/T/ipykernel_68879/2516972219.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(google_web_driver_path)
/var/folders/ly/yk3m2zx54dn3f8s2ydnw765r0000gn/T/ipykernel_68879/2516972219.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  reviews = driver.find_element_by_id("review")


In [3]:
book_reviews[:2]

['I really enjoy this novel. It has the things I enjoy reading about. Now I am typing more to fill the 140 character count that does not include spaces.',
 'story is good and mc is op in first chapters. But after 2100 chapters his attitude change and start begging and is polite to everyone. this kinda suck.\nand due to this i lost interest.']

# Training review data

In [4]:
import nltk 
from nltk.corpus import movie_reviews

In [5]:
positive_reviews =[]
for review in movie_reviews.fileids(categories='pos'):

    positive_reviews.append(movie_reviews.raw(review))
    
negative_reviews =[]
for review in movie_reviews.fileids(categories='neg'):

    negative_reviews.append(movie_reviews.raw(review))
 

In [6]:
print(len(positive_reviews), len(negative_reviews))

1000 1000


In [7]:
positive_labels = [1 for review in positive_reviews]
negative_labels = [0 for review in negative_reviews]

In [8]:
positive_reviews[:1]

['films adapted from comic books have had plenty of success , whether they\'re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there\'s never really been a comic book like from hell before . \nfor starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid \'80s with a 12-part series called the watchmen . \nto say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . \nthe book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . \nin other words , don\'t dismiss this film because of its source . \nif you can get past the whole comic book thing , you might find another stumbling block in from hell\'s directors , albert and allen hughes . \ngetting the hughes brothers to direct this see

In [9]:
from helper_function import cleanText, removeStopWords, lemmaText

In [10]:
for i in range(1000):
    positive_reviews[i] = lemmaText(removeStopWords(cleanText( positive_reviews[i])))
    negative_reviews[i] = lemmaText(removeStopWords(cleanText(negative_reviews[i])))

In [11]:
all_reviews = positive_reviews + negative_reviews

labels = positive_labels + negative_labels

In [12]:
positive_reviews_str = " ".join(positive_reviews)
negative_reviews_str = " ".join(negative_reviews)

reviews_str = positive_reviews_str +" " + negative_reviews_str

unique_words = set(reviews_str.split())

In [13]:
frequency =dict.fromkeys(unique_words,[0,0])

for word, value_pair in frequency.items():
    
    frequency[word] = [positive_reviews_str.count(word), negative_reviews_str.count(word)]

In [14]:
count = 0

for word, key in frequency.items():
    
    print(word, " : ", key)
    
    if count ==15:
        break
    count +=1

sabihy  :  [0, 1]
tidal  :  [2, 7]
tornado  :  [10, 9]
dust  :  [104, 72]
nobility  :  [6, 0]
nypd  :  [3, 4]
windowdresser  :  [0, 1]
teris  :  [23, 16]
uncooperative  :  [2, 0]
pseudoplaymate  :  [1, 0]
elvira  :  [1, 0]
verbalize  :  [0, 1]
torturously  :  [0, 1]
plumets  :  [0, 1]
headtohead  :  [3, 2]
darndest  :  [1, 0]


In [15]:
import pandas as pd

df = pd.DataFrame()

for review ,label in zip(all_reviews,labels):
    
    pos = 0
    neg = 0
    for word in review.split():
        
        pos += frequency[word][0]
        neg += frequency[word][1]
    data = {"bias":1, "positive": pos, "negative": neg, "sentiment": label}
    df= df.append(data, ignore_index=True)

In [16]:
df.head()

,bias,positive,negative,sentiment
0,1.0,394606.0,353652.0,1.0
1,1.0,226116.0,211206.0,1.0
2,1.0,125520.0,116157.0,1.0
3,1.0,813761.0,727562.0,1.0
4,1.0,344181.0,324640.0,1.0


# Logistic Regression Implementation

In [17]:
import warnings
warnings.filterwarnings('ignore')



class LogisticRegressionn:
    
    def __init__(self,alpha =0.0001, iteration= 1000000):
        
        self.alpha = alpha
        self.iteration = iteration
        self.weight = None
        
    def fit(self, X,y):
        
        m, n = X.shape
        
        self.weight = np.zeros(n)

        
        for _ in range(self.iteration):
            
            y_linear = np.dot(X, self.weight)
            
            h = self._sigmoid(y_linear)
            
            dw = (1 / m) * np.dot(X.T, (h - y))
            # update the weights theta
            self.weight -= self.alpha * dw
            
    def predict(self,x):
        
        y_linear = np.dot(x, self.weight)
        
        h = self._sigmoid(y_linear) 
        return [1 if i > 0.5 else 0 for i in h]
    
    def _sigmoid(self,x):
        
        return 1 /(1+ np.exp(-x))

In [18]:
import numpy as np
X = np.array(df[["bias","positive","negative"]])
y = np.array(df["sentiment"])




logistic = LogisticRegressionn()

logistic.fit(X,y)

z =logistic.predict(X)

In [19]:
for i in range(len(book_reviews)):
    book_reviews[i] = lemmaText(removeStopWords(cleanText(str(book_reviews[i]))))

In [20]:
book_reviews[:2]

['really enjoy novel thing enjoy reading type fill 140 character count include space',
 'story good mc op first chapter 2100 chapter attitude change start beg polite everyone kinda suckand due lose interest']

In [21]:
book_review_df =pd.DataFrame()


for review in book_reviews:
    
    pos = 0
    neg =0
    
    for word in review.split():
        
        if word in frequency.keys():
            pos += frequency[word][0]
            neg += frequency[word][1]
    data ={"bias": 1, "positive": pos,"negative": neg}
    book_review_df = book_review_df.append(data, ignore_index=True)
    

In [22]:
book_review_df['Sentiment'] = logistic.predict(np.array(book_review_df))

In [23]:
book_review_df

,bias,positive,negative,Sentiment
0,1.0,7490.0,6978.0,1
1,1.0,11344.0,10004.0,1
2,1.0,13542.0,12186.0,1
3,1.0,334059.0,309602.0,1
4,1.0,12634.0,10910.0,1
5,1.0,33028.0,27672.0,1
6,1.0,55750.0,54591.0,0
7,1.0,7864.0,6795.0,1
8,1.0,8145.0,7069.0,1
9,1.0,3891.0,4185.0,0


In [24]:
# Classifier said 6th index negative , lets look at it

book_reviews[6]

'im go honest cup tea enjoy idea super op bookas cheat still cant get fact mc keep meeting woman say heaven defy willpower cant even stop gawk asset full filler cant even begin describe sad thing filler doesnt contribute anything godamn advancement plot depress skip chapter hope dialogue end stop eye as overall able get stupid amount filler subpar mc doesnt live suppose personality follow certain xianxia plot may enjoy'

In [25]:
book_reviews[15]

'lv _ chapter good story author wait dont drop 3'

In [26]:
book_reviews[0]

'really enjoy novel thing enjoy reading type fill 140 character count include space'